In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import JavascriptException, NoSuchElementException, StaleElementReferenceException
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from selenium.common.exceptions import ElementNotInteractableException, ElementNotSelectableException
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
from IPython.display import clear_output
import os
import base64
from github import Github
from github import InputGitTreeElement
from datetime import datetime
import PyPDF2
from tika import parser
import pdfplumber

In [3]:
download_path = 'C:\\Users\\rlitt\\Documents\\GitHub\\apache'
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--verbose')
chrome_options.add_experimental_option("prefs", {
        "download.default_directory": download_path,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing_for_trusted_sources_enabled": False,
        "safebrowsing.enabled": False
})
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-software-rasterizer')


In [4]:
def get_new_pdf():
    #load page
    global driver
    print('getting pdf')
    driver = webdriver.Chrome(options=chrome_options, executable_path="chromedriver.exe")
    link_to_results_pdf = '/html/body/div[1]/div[4]/main/aside/p[2]/a'
    driver.get('https://www.apachecountyaz.gov/Elections')
    time.sleep(5)
    pdf_url = driver.find_element_by_xpath(link_to_results_pdf).get_attribute('href')
    driver.get(pdf_url)
    time.sleep(5)
    driver.find_element_by_xpath('/html/body/div[3]/div[3]/div/div[3]/div[2]/div[2]/div[3]/div').click()

def delete_pdf():
    pdf_object.close()
    os.remove(f'{download_path}\\ElectionResults.pdf')
    os.remove(f'{download_path}\\ElectionResults.txt')
    print('deleting old version')
    
def convert_pdf_to_text():
    global pdf_object
    print('converting to txt')
    pdf_object = pdfplumber.open(f'{download_path}\\ElectionResults.pdf')
    pdf_output = ''
    for i in range(3):
        pdf_page = pdf_object.pages[i]
        pdf_text = pdf_page.extract_text()
        pdf_output = pdf_output + pdf_text
    outfile = open(f'{download_path}\\ElectionResults.txt', "w")
    outfile.write(pdf_output)
    outfile.close()
    driver.close()

#cribbed from https://stackoverflow.com/questions/50071841/how-to-push-local-files-to-github-using-python-or-post-a-commit-via-python
def commit_new_pdf():
    print('pushing pdf')
    user = "ryanelittle"
    password = ""
    g = Github(user,password)
    repo = g.get_user().get_repo('apache')
    now = datetime.now()
    commit_message = f'refreshed update at {now.strftime("%H:%M:%S")}'
    master_ref = repo.get_git_ref('heads/master')
    master_sha = master_ref.object.sha
    base_tree = repo.get_git_tree(master_sha)
    element_list = list()
    with open(f'{download_path}\\ElectionResults.txt') as input_file:
        data = input_file.read()
    element = InputGitTreeElement('apache-election-results.txt', '100644', 'blob', data)
    element_list.append(element)
    tree = repo.create_git_tree(element_list, base_tree)
    parent = repo.get_git_commit(master_sha)
    commit = repo.create_git_commit(commit_message, tree, [parent])
    master_ref.edit(commit.sha)
    print(f'pdf pushed at {now.strftime("%H:%M:%S")}')

In [ ]:
count = 0
for i in range(10000000):
    count = count + 1
    print(f'Starting loop {str(count)}.')
    get_new_pdf()
    time.sleep(5)
    try:
        convert_pdf_to_text()
        time.sleep(10)
        commit_new_pdf()
        time.sleep(15)
        delete_pdf()
    except:
        pass
    time.sleep(30)
    clear_output(wait=True)

Starting loop 1403.
getting pdf
